In [ ]:
from Update_MC.download_SSI_Price import get_data_SSI_price
# data = get_data_SSI_price("HOSE", "Update_MC")

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from Methods.base import Base, convert_strF_to_arrF, calculate_formula

df_CT = "Update_MC/Top_100_GeoPro_VN_2023_35Fs.csv"
if type(df_CT) == str:
    df_CT = pd.read_csv(df_CT)

data_new_MC = pd.read_csv("Update_MC/_price_ssi_temp_/EXCHANGE_HOSE.csv", skiprows=[0])[["CK", "TC", "Tổng KL"]]
data_new_MC = data_new_MC[data_new_MC["Tổng KL"].notna()]
data_new_MC["MARKET_CAP"] = data_new_MC["TC"] * data_new_MC["Tổng KL"]
data = pd.read_excel("HOSE_File3_2023_Field.xlsx")
data_old_MC = pd.read_excel("Update_MC/2022_HOSE.xlsx")[["Symbol", "MARKET_CAP"]]
data_old_MC = data_old_MC[data_old_MC["MARKET_CAP"].notna()]

In [ ]:
list_symbol = np.intersect1d(data["SYMBOL"][data["TIME"]==2023], data_old_MC["Symbol"])
list_symbol = np.intersect1d(list_symbol, data_new_MC["CK"])
new_MC = np.array(data_new_MC.set_index("CK").loc[list_symbol, "MARKET_CAP"])
old_MC = np.array(data_old_MC.set_index("Symbol").loc[list_symbol, "MARKET_CAP"])
new_data = data[(data["TIME"]==2023) & (data["SYMBOL"].isin(list_symbol))].copy()
new_data = new_data.set_index("SYMBOL").loc[list_symbol].reset_index()

In [ ]:
vis = Base(new_data)
def get_weight_array(formula):
    if type(formula) == str:
        formula = convert_strF_to_arrF(formula)
    
    weight = calculate_formula(formula, vis.OPERAND)
    return weight

In [ ]:
list_invest = []
for ct in df_CT["CT"]:
    weight = get_weight_array(ct)
    start, end = vis.INDEX[0], vis.INDEX[1]
    arr_max = np.where(weight[start:end] == max(weight[start:end]))[0]
    if arr_max.shape[0] == 1:
        cty = vis.symbol_name[vis.SYMBOL[arr_max[0]]]
    else:
        cty = "NotInvest"
    
    list_invest.append(cty)

Counter(list_invest)